# Ricerca: esercizi su espressioni regolari

Se non dobbiamo compiere operazioni *veramente* molto semplici i metodi delle stringhe (`replace`, `search`, `index`, `upper`, `lower`, etc...) avrai bisogno di scrivere molto codice, lungo da leggere e da scrivere.

Se il tuo problema non è banale allora sicuramente avrai giovamento nell'utilizzare delle *espressioni regolari*.


## 1 Prendiamoci i dati

Per partire da casi concreti, come già fatto in precedenza andiamo a cercarci dei dati dal catalogo opendata `dati.trentino.it`. In questo caso sceglieremo un file dal dataset [ Trasporti pubblici del Trentino (formato GTFS) ](http://dati.trentino.it/dataset/trasporti-pubblici-del-trentino-formato-gtfs). 

**DOMANDA 1.1**: Quale è la licenza del dataset? Possiamo farci tutto quello che vogliamo ? 

Il [formato GTFS](https://developers.google.com/transit/gtfs/) è un pratico formato per gli orari e tracciati del trasporto pubblico. Questo formato ci descrive i campi che ci aspettiamo nei file. Ma i file, fisicamente, in che formato sono?

Nel dataset troviamo la risorsa [GTFS Urbano TTE](http://dati.trentino.it/dataset/trasporti-pubblici-del-trentino-formato-gtfs/resource/57869023-adfa-467e-8100-76403257d2d1) che al suo interno contiene un link  [ad uno zip](http://www.ttesercizio.it/opendata/google_transit_urbano_tte.zip). Se apriamo lo zip troveremo diversi `.txt` che se attentamente osservati rivelano essere in formato CSV (cominciate a notare l'utilità del formato ;-) ?


Concentriamoci sul file [stops.txt](stops.txt), di cui vediamo un estratto qui:

```
stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,wheelchair_boarding
1,28105z,Baselga Del Bondone,,46.078317,11.046924,10110,2
2,28105x,Baselga Del Bondone,,46.078581,11.047541,10110,2
3,27105c,Belvedere,,46.044406,11.105342,10110,2
4,22220z,Lamar Ponte Avisio,,46.134620,11.110914,10110,2
5,28060z,Sp 85 Bivio Sopramonte,,46.085226,11.069313,10110,2
7,24405z,Maso Bolleri,,46.102485,11.124174,10110,2
8,24405x,Maso Bolleri,,46.102234,11.123940,10110,2
9,25205x,Borino,,46.067367,11.165050,10110,2
```

Come ci aspettiamo da un buon file CSV, nella prima riga costituisce le intestazioni e vediamo che i campi sono separati da virgole. 

**DOMANDA 1.2**: Dove possiamo trovare il significato del file ?


## 2. Verifichiamo che i dati siano corretti

Come avrai intuito, il file `stops.txt` è un file CSV che contiene le informazioni riguardanti le stazioni degli autobus di Trento. 

Supponiamo che tu voglia fare avere a lista di tutte le fermate su *strade provinciali* (sigla *SP*): 

**DOMANDA 2.1**: Guarda bene dentro il [file completo](stops.txt): i dati sono sempre perfettamente regolari come ci piacerebbe? 

**DA FARE 2.2**: Con quello che sai [dalla lezione 2](http://softpython.readthedocs.io/it/latest/exercises/data-formats/data-formats-exercises.html#File-CSV), prova ad aprire il CSV e stampare solo le linee che contengono qualcosa che assomiglia a strade provinciali. 

**NOTA**: non serve che filtri per bene tutte, fai solo qualche tentativo per i casi più ovvi, usando funzioni sulle stringhe che già conosci. Ai casi più diccifili ci penseremo in seguito con le regex! 

**SUGGERIMENTO 1**: usa il metodo `upper` delle stringhe:

In [1]:
"Ciao MONDO".upper()

'CIAO MONDO'

In [2]:
"SoftPython".upper()

'SOFTPYTHON'

In [3]:
"ab cde".find('cd')

3

In [4]:
"ab cde".find(' c')

2

In [5]:
"ab cde".find('a')

0

In [6]:
"ab cde".find('D')

-1

In [7]:
"ab cde".find('z')

-1

Il risultato probabilmente non ti entusiasmerà, perchè `SP` si presenta in tante parole, e anche quando è riferito alle superstrade a volte è puntato e a volte no. Proviamo a vedere cosa sono queste **espressioni regolari**:

> **WIKIPEDIA**
>
> Una *espressione regolare* (in lingua inglese regular expression o, in forma abbreviata, regexp, regex o RE) è una sequenza di simboli (quindi una stringa) che identifica un insieme di stringhe: essa definisce una funzione che prende in ingresso una stringa, e restituisce in uscita un valore del tipo sì/no, a seconda che la stringa segua o meno un certo *pattern*.

L'utilizzo di una espressione regolare è sicuramente più veloce perché ci permette di cercare non solo una stringa bensì un intero insieme di stringhe, detto appunto *pattern*. Una considerazione da fare è che nonostante i concetti
riguardo le regex siano universali, alcune implementazioni si differenziano nel comportamento in alcuni casi particolari oppure aggiungendo funzionalità *non standard*.


## 3. Creiamoci una funzione di test

In ciò che segue, ci piacerebbe testare varie _regex_ su delle stringhe di prova. Dato che scrivere i comandi per stampare i risultati dei test può diventare ripetitivo e noiso, ci conviene creare una _funzione_  con dentro delle istruzioni di stampa da eseguire automaticamente (per più info sulle funzioni, vedi [capitolo 3 Pensare in Python ](https://davidleoni.github.io/ThinkPythonItalian/html/thinkpython2004.html)):

In [8]:
import re

def test_regex(pattern):
    names = ['Baselga del Bondone',
             'Spini Bregenz', 
             'Sp.89 Maso Brentegam', 
             'sp 90 "Maso Prudenza"', 
             'Brancolino Sp.90']
    for name in names:
        print(re.search(pattern, name, re.I))
    
test_regex("SP")

None
<_sre.SRE_Match object; span=(0, 2), match='Sp'>
<_sre.SRE_Match object; span=(0, 2), match='Sp'>
<_sre.SRE_Match object; span=(0, 2), match='sp'>
<_sre.SRE_Match object; span=(11, 13), match='Sp'>



Ignoriamo per il momento `import re`, che semplicemente importa la libreria per le regex. 

In Python, le funzioni si dichiarano con la parola riservata `def` seguita dal nome della funzione che scegliamo arbitrariamente noi.  In questo caso il nome scelto è `test_regex`:

```python
def test_regex(pattern):
```
Poi, a questa funzione abbiamo deciso che bisognerà passare un parametro, che chiamiamo `pattern` (ma potremmo chiamarlo come ci pare, anche `pippo`).

**NOTA**: alla fine della prima riga, ci sono dei due punti `:` se dimentichi di metterli potresti trovarti con strani errori di syntassi ! 

La nostra funzione farà qualcosa con questa variabile che abbiamo chiamato `pattern`:

```python
def test_regex(pattern):
    names = ['Baselga Del Bondone',
             'Spini Bregenz',
             'Sp.89 Maso Brentegam', 
             'sp 90 "Maso Prudenza"', 
             'Brancolino Sp.90']
    for name in names:
        print(re.search(pattern, name, re.I))
```

In questo caso, per ciascun nome, eseguirà questa riga (vedremo in seguito il contenuto della `print`):

```python
print(re.search(pattern, name, re.I))
```

sfruttando la variabile `pattern` che passeremo al momento di chiamare la funzione: 



In [9]:
test_regex("SP")

None
<_sre.SRE_Match object; span=(0, 2), match='Sp'>
<_sre.SRE_Match object; span=(0, 2), match='Sp'>
<_sre.SRE_Match object; span=(0, 2), match='sp'>
<_sre.SRE_Match object; span=(11, 13), match='Sp'>



**DA FARE 3.1**: Copia a mano la funzione di sopra qua sotto, ed eseguila con Control + Invio:

In [10]:
# scrivi qua

**DA FARE 3.2**: Cosa significa l'opzione `re.I` nella chiamata a `re.search`? Riesci a capire cos'è? Prova a copiare la funzione qua sotto chiamandola `test_regex_no_i` e togliere la `re.I`. Se ancora non capisci usa la Forza (anche `help(re)` può funzionare).

In [11]:
#def test_regex_no_i(pattern):
    # scrivi qua
    


### La prima regex

È ora di usare la nostra prima regex, per farlo dobbiamo guardare le differenze tra le stringhe corrette e quella errata: in questo caso sappiamo che ogni *strada provinciale* ha un numero. Proviamo con questa regex:


In [12]:
test_regex(r"sp.\d\d")

None
None
<_sre.SRE_Match object; span=(0, 5), match='Sp.89'>
<_sre.SRE_Match object; span=(0, 5), match='sp 90'>
<_sre.SRE_Match object; span=(11, 16), match='Sp.90'>


Ora ti chiederai che cosa significa questa stringa, andiamo con ordine:

- La `r` prima dell'inizio della stringa serve ad indicare a Python che la seguente è una 
raw string, cioè una stringa in cui non deve espandere le *sequenze di escape* (cioè `\` seguito da altri caratteri al fine di generare caratteri non stampabili, per esempio `\n` è il carattere di new-line).

**DA FARE 3.3**: magari già conosci le _sequenze  di escape_, si trovano in molti linguaggi. Se non le conosci, prova a scrivere i comandi qua sotto, sempre in nuove celle:

* `print("ciao mondo")`
* `print("ciao\tmondo")`
* `print("ciao\nmondo")`
* `print("ciao\rmondo")` (questo è strano...)
* `print("ciao\\mondo")`

Che differenze noti? E se metti il carattere `r` _davanti_ alle stringhe (quindi subito prima del doppio apice `"`, come in `r"hello"`), che succede ?



In [13]:
# prova qua


Tornando alla regex  `r"sp.\d\d"`
- Abbiamo visto che le lettere dell'alfabeto (e i numeri) hanno semplicemente il loro valore.
- Il carattere `.` in questo caso è un *metacarattere* che in questo caso si comporta come un "jolly" e può identificare qualsiasi carattere ad eccezione de carattere di fine riga (solitamente).
- `\d` sono due caratteri ma ai fini della regex sono da considerarsi uno solo. Ogni volta che vediamo il carattere `\` è da considerarsi assieme al carattere successivo. Il significato in questo caso è una qualsiasi cifra tra `0` e `9`.

---

**DA FARE 3.4**: Prova ad integrare questo esempio con quello precedente: copia il primo esempio qui sotto, aggiungi la libreria `re` e modifica *linea 7* perché l'*if* utilizzi la regular expression come abbiamo visto nell'esempio precedente.


In [14]:

# Copia il codice qui


## 4. Sintassi delle Python RegEx

Proviamo ora a guardare alcuni meta-caratteri importanti nelle regular expression in Python - (scusate per l'orribile tabella ma fare tabelle in Jupyter è [sempre problematico](https://github.com/jupyter/notebook/issues/3024):

### Meta-caratteri

| Modificatore | Descrizione |
|:---:|:---:|
| `\` | usato come *escape* (cioè segnalare che il carattere a seguire, nonostante sia un carattere speciale deve essere trattato come se non lo fosse) o per iniziare una *sequenza* (vedi sotto). |
| `.` | usato per rappresentare un qualsiasi carattere ad eccezione della nuova linea (con l'opzione `re.A` possiamo rimuovere anche questa eccezione). |
| `^` | usato per indicare l'inizio della riga |
| `$` | usato per indicare la fine della riga |
| `[...]` | usate per racchiudere l'insieme di caratteri che verificano questa espressione regolare |
| `[^...]` | usate per racchiudere l'insieme di caratteri che se presenti **NON** verificano questa espressione regolare |
| `A` &#124; `B` | usato per indicare una rappresentazione alternativa, è valida sia che appaia `A`, sia che appaia `B` |
| `()` | usate come in matematica per indicare la precedenza sulle operazioni |

Nel codice qui sotto definiamo la funzione `test_regex_num()` che è come quella già ma vista, ma usando dei numeri di telefono al posto dei nomi delle fermate; per ogni numero controlliamo se l'espressione regolare viene soddisfatta e se lo è lo stampiamo ed in fine testiamo varie proprietà di questi numeri di telefono.

In [15]:
import re
def test_regex_num(pattern):
    numbers = ['3471234567', 
             '3303303367', 
             '3232123323', 
             '3383123222']
    for num in numbers:
        if re.search(pattern, num):
            print(num)
    print("-----")

print("Tutti i numeri che contengono 33")
test_regex_num("33")
print("Tutti i numeri che iniziano per 33")
test_regex_num("^33")
print("Tutti i numeri che hanno come penultima cifra 2")
test_regex_num("2.$")
print("Tutti i numeri che contengono 212 o 312")
test_regex_num("212|312")
# Oppure
test_regex_num("[23]12")
# Oppure
test_regex_num("(2|3)12")


Tutti i numeri che contengono 33
3303303367
3232123323
3383123222
-----
Tutti i numeri che iniziano per 33
3303303367
3383123222
-----
Tutti i numeri che hanno come penultima cifra 2
3232123323
3383123222
-----
Tutti i numeri che contengono 212 o 312
3232123323
3383123222
-----
3232123323
3383123222
-----
3232123323
3383123222
-----


**DA FARE 4.1** Prova a scrivere qua sotto i pattern che soddisfano le proprietà richieste nelle chiamate a `test_regex_num`. Cerca di *non guardare* all'esercizio precedente:

In [16]:

def test_regex_num(pattern):
    numbers = ['3471234567', 
             '3303303367', 
             '3232123323', 
             '3383123222']
    for num in numbers:
        if re.search(pattern, num):
            print(num)
    print("-----")

print("Tutti i numeri che contengono 32")
test_regex_num("")    # metti il pattern giusto

print("Tutti i numeri che finiscono per 67")
test_regex_num("")

print("Tutti i numeri che hanno come quarta cifra 3")
test_regex_num("")

print("Tutti i numeri che contengono 232 o 233 o 234")
test_regex_num("")
# Oppure
test_regex_num("")
# Oppure
test_regex_num("")

Tutti i numeri che contengono 32
3471234567
3303303367
3232123323
3383123222
-----
Tutti i numeri che finiscono per 67
3471234567
3303303367
3232123323
3383123222
-----
Tutti i numeri che hanno come quarta cifra 3
3471234567
3303303367
3232123323
3383123222
-----
Tutti i numeri che contengono 232 o 233 o 234
3471234567
3303303367
3232123323
3383123222
-----
3471234567
3303303367
3232123323
3383123222
-----
3471234567
3303303367
3232123323
3383123222
-----


### Ripetizioni

Le espressioni regolari possono anche gestire delle ripetizioni di particolari pattern utilizzando altri caratteri speciali. 

| Modificatori | Descrizione |
| :---: | :--- |
| `{m, n}` | il carattere o gruppo a cui è riferito viene ripetuto almeno `m` volte fino ad un massimo di `n` volte. |
| `{m}` | il carrattere o il gruppo a cui è riferito viene ripetuto esattamente `m` volte |
| `?` | il carattere o il gruppo a cui è riferito viene ripetuto `0` o `1` volta. Equivale a `{,1}`. |
| `*` | il carattere o il gruppo a cui è riferito viene ripetuto `0` o più volte . Equivale a `{,}`. |
| `+` | il carattere o il gruppo a cui è riferito viene ripetuto `1` o più volte. Equivale a `{1,}`. |

I caratteri o gruppi a cui si riferiscono i `modificatori delle ripetizioni` appena precedenti ad essi, vediamo un esempio

In [17]:
print(re.search("a+b", "aaaabbb"))
print(re.search("a+b", "bab"))
print(re.search("a+b", "bbb"))
print(re.search("a+b", "aaaa"))

<_sre.SRE_Match object; span=(0, 5), match='aaaab'>
<_sre.SRE_Match object; span=(1, 3), match='ab'>
None
None


Come pui vedere nell'esempio qui sopra il pattern `a+b` indica `a` una o più volte, seguito da una `b`. Quano si ha un match puoi vedere nell'oggetto ritornato dal metodo `re.search()` la sottostringa che ha verificato la regex (usando il metodo `group()`) e gli indici della posizione di essa all'interno della stringa (chiamando il metodo `.span()`).

### Funzione `pass_n_fail`


Qui sotto la funzione `pass_n_fail()` prende come parametri un `pattern` e due liste di stringhe: `pass_list` e `fail_list`. La funzione verifica se quelle che appartengono alla prima lista matchano l'espressione regolare mentre quelle nella seconda non la matchano:

In [18]:
def pass_n_fail(pattern, pass_list, fail_list):
    for p in pass_list:
        if not re.search(pattern, p):
            print("ERRORE: '{}' non matcha il pattern '{}' ma dovrebbe farlo!".format(p, pattern))
    for f in fail_list:
        if re.search(pattern, f):
            print("ERRORE: '{}' matcha il pattern '{}' ma non dovrebbe farlo!".format(f, pattern))


Guardiamo quest'esempio:

In [19]:

# ESEMPIO
pass_n_fail("c",
            ["aa","a"],  # espressioni che vorremmo matchassero il pattern "c"
            ["b","c"]    # espressioni che vorremmo NON matchassero il pattern "c"
           )

ERRORE: 'aa' non matcha il pattern 'c' ma dovrebbe farlo!
ERRORE: 'a' non matcha il pattern 'c' ma dovrebbe farlo!
ERRORE: 'c' matcha il pattern 'c' ma non dovrebbe farlo!


Quest'esempio qua invece non dovrebbe darci nessun output, perchè le stringhe `"aa"` che `"a` matchano il pattern `"a"`, e  le stringhe `"b"` e `"c"` non matchano il pattern `"a"`:

In [20]:

# ESEMPIO
pass_n_fail("a",
            ["aa","a"],  # espressioni che vorremmo matchassero il pattern "a"
            ["b","c"]    # espressioni che vorremmo NON matchassero il pattern "a"
           )



**DA FARE 4.2**: Prova tu a fornire esempi nelle due liste qua sotto (non preoccuparti se vedi scritto `ERRORE` nell'output della cella prima ancora di cominciare, se metti esempi giusti ed esegui la cella i messaggi di errore dovrebbero sparire):

In [21]:
pass_n_fail("a",
            ["",""], # metti esempi che matchano
            ["",""]  # metti esempi che non matchano
           )

ERRORE: '' non matcha il pattern 'a' ma dovrebbe farlo!
ERRORE: '' non matcha il pattern 'a' ma dovrebbe farlo!


In [22]:
 # Come sopra, ma un po più difficile:
pass_n_fail("ab+",
            ["",""], 
            ["",""]
           )

ERRORE: '' non matcha il pattern 'ab+' ma dovrebbe farlo!
ERRORE: '' non matcha il pattern 'ab+' ma dovrebbe farlo!


In [23]:
# Come sopra, un po più difficile ancora:
pass_n_fail("ab*",
            ["",""], 
            ["",""]
           )

ERRORE: '' non matcha il pattern 'ab*' ma dovrebbe farlo!
ERRORE: '' non matcha il pattern 'ab*' ma dovrebbe farlo!


In [24]:
# OK! Ancora un paio
pass_n_fail("^[ab]+[^ab]$",
            ["",""],
            ["",""]
           )

ERRORE: '' non matcha il pattern '^[ab]+[^ab]$' ma dovrebbe farlo!
ERRORE: '' non matcha il pattern '^[ab]+[^ab]$' ma dovrebbe farlo!


In [25]:
# L'ultima
pass_n_fail(".?\.{3}$",
            ["",""], #TODO
            ["",""]
           )

ERRORE: '' non matcha il pattern '.?\.{3}$' ma dovrebbe farlo!
ERRORE: '' non matcha il pattern '.?\.{3}$' ma dovrebbe farlo!


## 5. Sequenze

A volte vogliamo considerare insiemi molto grandi di possibili simboli in una espressione regolare: per esempio se vogliamo validare la struttura di un **indirizzo email** vogliamo controllare che

* contenga almeno 3 caratteri
* una `@`
* altri 3 caratteri 
* un punto 
* e almeno altri 2 caratteri.

Il problema è che alcuni caratteri non possono essere presenti nelle email (come ad esempio `\|{}()[]` etc...), e se dovessimo scrivere un set di caratteri da escludere usando l'espressione `[^...]` ci costerebbe molto tempo e spazio, inoltre sarebbe facile dimenticarsi qualche simbolo e quasi impossibile da leggere.

Per ovviare a questo problema sono stati introdotte delle scorciatoie: delle *sequenze* di simboli che vanno a sostituire lunghi set di caratteri di comune utilizzo, eccone alcuni:

| Sequenza | Descrizione |
| :---: | :--- |
| `\A` | Inizio della stringa (simile a `^`) |
| `\b` | Valida la stringa vuota che delimita una parola |
| `\d` | Cifre da 0 a 9 |
| `\D` | Tutto eccetto le cifre |
| `\s` | Spaziature |
| `\S` | Tutto eccetto le spaziature |
| `\w` | Tutti i caratteri alfanumerici e `_` |
| `\W` | Tutto eccetto i caratteri alfanumerici e l'underscore `_` |
| `\Z` | Fine della stringa (simile a `$`) |

**DA FARE 5.1**: Proviamo a variare l'esercizio precedente adesso quando matcherà sarai tu a *scrivere dei pattern* che verifichi le stringhe nella `pass_list` ed escluda quelle nella `fail_list`, dove troverai il pattern sarà come l'esercizio precedente (di nuovo non preoccuparti se vedi 'ERRORE' scritto sotto le celle prima ancora di iniziare, se metti i giusti pattern / esempi come richiesto le scritte 'ERRORE' dovrebbero scomparire )


In [26]:
#Scegli il PATTERN giusto !
pass_n_fail(r"",
            ["3 ramarri", "2 carri"],  # queste devono matchare
            ["tre ramarri", "due carri", "3 ", "2 "]  # queste non devono matchare
           )

ERRORE: 'tre ramarri' matcha il pattern '' ma non dovrebbe farlo!
ERRORE: 'due carri' matcha il pattern '' ma non dovrebbe farlo!
ERRORE: '3 ' matcha il pattern '' ma non dovrebbe farlo!
ERRORE: '2 ' matcha il pattern '' ma non dovrebbe farlo!


In [27]:
#Scegli le STRINGHE
pass_n_fail(r"^(w{3}\.)?\w\w+\.\w\w+",
            ["",""],
            ["",""]
           )

ERRORE: '' non matcha il pattern '^(w{3}\.)?\w\w+\.\w\w+' ma dovrebbe farlo!
ERRORE: '' non matcha il pattern '^(w{3}\.)?\w\w+\.\w\w+' ma dovrebbe farlo!


In [28]:
#Scegli il PATTERN
pass_n_fail(r"",
            ["21.12.2017", "11/01/2018", "16-12-89"],
            ["1621211003", "11/20/2010", "12-12-123"]
           )

ERRORE: '1621211003' matcha il pattern '' ma non dovrebbe farlo!
ERRORE: '11/20/2010' matcha il pattern '' ma non dovrebbe farlo!
ERRORE: '12-12-123' matcha il pattern '' ma non dovrebbe farlo!


## 6. Le funzioni della libreria `re`

Fino ad ora abbiamo usato una sola funzione della libreria `re` di Python, ossia `re.search()` ma sono presenti anche altre funzionalità, la più simile è il metodo `re.match()`:


In [29]:
print(re.search('c', 'abcde'))

<_sre.SRE_Match object; span=(2, 3), match='c'>


In [30]:
print(re.match('c', 'abcde'))

None


In [31]:
print(re.match('a', 'abcde'))

<_sre.SRE_Match object; span=(0, 1), match='a'>


**DA FARE 6.1**:  Riesci a capire la differenza? `help(re.match)` e `help(re.search)` possono tornarti utili. 

## 7. Sostituzioni con `re.sub`
Le espressioni regolari possono anche essere utilizzate per sostituire del testo, un po' come il metodo `replace()` sulle stringhe. Quando chiamiamo il metodo `re.sub()`, per ricevere in output la stringa elaborata, dobbiamo passare come argomenti:

1. il pattern della regular expression
2. il testo da sostituire
3. la stringa su cui effettuare la ricerca

Tornando all'esempio delle fermate dell'autobus, adesso vogliamo sostituire tutto quel confusionario "Sp" in "strade provinciali", vediamo il codice:

In [32]:
import csv
with open('stops.txt', newline='') as f:
    reader = csv.reader(f, delimiter=',')    
    for row in reader:
        row[2] = re.sub(r'Sp.(\d+)', r'Strada Provinciale \1', row[2])
        print(row)

['stop_id', 'stop_code', 'stop_name', 'stop_desc', 'stop_lat', 'stop_lon', 'zone_id', 'wheelchair_boarding']
['1', '28105z', 'Baselga Del Bondone', '', '46.078317', '11.046924', '10110', '2']
['2', '28105x', 'Baselga Del Bondone', '', '46.078581', '11.047541', '10110', '2']
['3', '27105c', 'Belvedere', '', '46.044406', '11.105342', '10110', '2']
['4', '22220z', 'Lamar Ponte Avisio', '', '46.134620', '11.110914', '10110', '2']
['5', '28060z', 'Strada Provinciale 85 Bivio Sopramonte', '', '46.085226', '11.069313', '10110', '2']
['7', '24405z', 'Maso Bolleri', '', '46.102485', '11.124174', '10110', '2']
['8', '24405x', 'Maso Bolleri', '', '46.102234', '11.123940', '10110', '2']
['9', '25205x', 'Borino', '', '46.067367', '11.165050', '10110', '2']
['10', '28205z', 'Cadine Strada Gardesana', '', '46.088630', '11.065018', '10110', '2']
['11', '28205x', 'Cadine Strada Gardesana', '', '46.088729', '11.064509', '10110', '2']
['12', '22110c', 'Canova Paludi', '', '46.099170', '11.109314', '10110

Bene, ma cosa succede? Conosciamo già tutto fino alla linea 3, poi cerchiamo il pattern `Sp.(\d+)` ; nota che ho messo delle parentesi dove sembrerebbe non servano ma fai attenzione al prossimo parametro (cioè la stringa sostitutiva): ti accorgerai di un carattere speciale di una sequenza particolare di cui non abbiamo parlato, ovvero `\1` . Questo tipo di sequenza è detta **gruppo di backreference** e viene definito nel pattern da cercare utilizzando le parentesi tonde appunto: tutto quello tra `(` and `)` diventa un gruppo e per richiamarlo è necessario soltando aggiungere un *backslash* seguito dal numero del gruppo, contando da sinistra a destra. 

Nel nostro caso esiste un solo gruppo quindi non possiamo sbagliare: la ricerca prima esamina il testo per trovare `Sp` seguito da un carattere e poi (da un gruppo definito come) una o più cifre. La stringa di sostituzione riporterà prima la scritta `Strada Provinciale` poi uno spazio ed infine la cifra che ha verificato il gruppo definito nel pattern di ricerca.

**DA FARE 7.1**: Prova a ricopiare il codice precedente qua sotto, e compi la stessa operazione ma questa volta con le *Strade Statali*. Riesci a scrivere pattern diversi di ricerca e sostituzione per farlo?

In [33]:
#Metodo 1 qui sotto

        
#Metodo 2 qui sotto


**NOTA**
In realtà l'utilizzo delle backreferences non è limitato alla funzione di sostituzione `re.sub` ma può essere usato anche nei pattern: il pattern `^(a+)=\1$` per esempio significa che la stringa deve iniziare `^` con una o più `a` ( `a+` ), e deve essere seguito da un uguale ( `=` ) e dalla stessa stringa che ha verificato il *gruppo 1* ( `\1` ) che nel nostro caso equivale a quanto stato verificato da `a+` precedentemente.

**DA FARE 7.2**
Prova a inserire delle stringhe che verificano i pattern inseriti:

In [34]:
pass_n_fail(r'^(a+)=\1$',
           ['', '',''], #TODO Inserisci almeno 3 elementi
           [])

ERRORE: '' non matcha il pattern '^(a+)=\1$' ma dovrebbe farlo!
ERRORE: '' non matcha il pattern '^(a+)=\1$' ma dovrebbe farlo!
ERRORE: '' non matcha il pattern '^(a+)=\1$' ma dovrebbe farlo!


## 8. Dividere le stringhe

Uno degli altri task ai quali le espressioni regolari possono essere utile è quello di spezzare le stringhe. Questo può essere utile ad esempio quando vogliamo accedere a specifiche informazioni oppure quando vogliamo analizzare del testo. Anche in questo caso esiste un'analogia con il metodo `split()` per le stringhe visto che entrambe le istruzioni compiono una funzione simile ma `re.split()` ha due parametri obbligatori, il primo è il `pattern` dell'espressione regolare che quando verificata spezza il testo e il secondo è la stringa da spezzare.

Nell'esempio qui sotto separare tutte le frasi in un testo, prendiamo il file, lo apriamo e lo leggiamo: `f.readlines()` legge tutte le linee e restituisce una lista di linee, il metodo `"\n".join()` usa il carattere di nuova riga `\n` per unire gli elementi della lista (in questo caso le varie linee, formando il testo).

A questo punto chiamiamo il metodo `re.split()` usando come pattern di separazione punti, punto e virgola, due punti, etc... presenti almeno una volta e seguiti da almeno uno spazio seguito da opzionalmente da una serie di acapo; abbiamo anche aggiunto il flag `re.MULTILINE` per permettere le regex su righe multiple.

In [35]:
testo = ""
with open('psposi1.txt') as f:
    testo = "\n".join(f.readlines())

re.split(r'[.;:?!-]+\s+\n*', testo, flags=re.MULTILINE)

['Quel ramo del lago di Como, che volge a mezzogiorno, tra due catene [1] non interrotte di monti, tutto a seni e a golfi, a seconda dello sporgere e del rientrare di quelli, vien, quasi a un tratto, a ristringersi, e a prender corso e figura di fiume, tra un promontorio a destra, e un’ampia costiera dall’altra parte',
 'e il ponte [2], che ivi congiunge le due rive, par che renda ancor più sensibile all’occhio questa trasformazione, e segni il punto in cui il lago cessa, e l’Adda rincomincia, per ripigliar poi nome di lago dove le rive, allontanandosi di nuovo, lascian l’acqua distendersi e rallentarsi in nuovi golfi e in nuovi seni',
 'La costiera, formata dal deposito di tre grossi torrenti [3], scende appoggiata a due monti contigui, l’uno detto di san Martino, l’altro, con voce lombarda, il Resegone, dai molti suoi cocuzzoli in fila, che in vero lo fanno somigliare a una sega',
 'talché non è chi, al primo vederlo, purché sia di fronte, come per esempio di su le mura di Milano che

## 9. Cercare pattern nel testo

A volte vogliamo trovare delle informazioni all'interno del testo e sappiamo che queste appaiono in un certo pattern, possiamo scrivere una espressione regolare e utilizzare il metodo `re.findall()` per estrarle.
Questo comando è molto simile al comando `re.find()` ma al contrario di questo non si ferma al primo match ma prosegue estraendo una lista delle stringhe che hanno verificato la regex oppure, se sono presenti delle backreferences, con *una lista di tuple avente i valori di tutti i gruppi di backreference* che hanno verificato il pattern.

Proviamo a cercare tutte le parole che seguono la parola `"terra"` o `"terre"` : la regular expression cerca prima una stringa che inizi per *terr* e che abbia una *a* oppure una *e*, uno spazio ed infine una stringa alfanumerica di uno o più caratteri ed un delimitatore di fine parola:

In [36]:
re.findall(r'terr[ae]\s\w+\b', testo)

['terre accennate', 'terra cotta', 'terra con']

Come vedi ci viene restituita una lista di stringhe, infatti non ci sono gruppi all'interno della regex e quindi ci viene restituito tutto il matching. 

**DA FARE 9.1**: Prova a copiare la linea di sopra qua sotto e ad aggiungere uno o più gruppi nel pattern e vedi cosa succede: